In [ ]:
import cv2
import torch
import torchvision.transforms as transforms
from PIL import Image
import os
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

input_video = "raw-traffic-footage/bridge-1.mp4"

frames_dir = "extracted_frames"
os.makedirs(frames_dir, exist_ok=True)

video = cv2.VideoCapture(input_video)

frame_rate = video.get(cv2.CAP_PROP_FPS)
frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

output_video = "output_video.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
output = cv2.VideoWriter(output_video, fourcc, frame_rate, (frame_width, frame_height))

frame_count = 0

while True:
    ret, frame = video.read()
    
    if not ret:
        break
    
    frame_path = os.path.join(frames_dir, f"frame_{frame_count:05d}.jpg")
    cv2.imwrite(frame_path, frame)
    
    frame_count += 1

video.release()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = fasterrcnn_resnet50_fpn(weights=None)

in_features = model.roi_heads.box_predictor.cls_score.in_features

num_classes = 2  
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

model.load_state_dict(torch.load("trained_model.pth"))
model.to(device)  
model.eval()

for frame_file in sorted(os.listdir(frames_dir)):
    frame_path = os.path.join(frames_dir, frame_file)
    
    image = Image.open(frame_path).convert("RGB")
    
    transform = transforms.Compose([
        transforms.Resize((480, 480)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image = transform(image)
    
    image = image.unsqueeze(0).to(device)  
    
    with torch.no_grad():
        outputs = model(image)
    
    boxes = outputs[0]['boxes'].cpu().numpy()
    labels = outputs[0]['labels'].cpu().numpy()
    scores = outputs[0]['scores'].cpu().numpy()
    
    print(f"Frame: {frame_file}")
    print(f"Boxes: {boxes}")
    print(f"Labels: {labels}")
    print(f"Scores: {scores}")
    
    frame = cv2.imread(frame_path)
    height, width, _ = frame.shape
    for box, label, score in zip(boxes, labels, scores):
        if score > 0.05:  
            x1, y1, x2, y2 = box
            x1 = int(x1 * width)
            y1 = int(y1 * height)
            x2 = int(x2 * width)
            y2 = int(y2 * height)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"Label: {label}, Confidence: {score:.2f}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    output.write(frame)

output.release()

import shutil
shutil.rmtree(frames_dir)

print("Video processing completed.")